In [1]:
import pickle
import pandas as pd
import numpy as np
import torch
import transformers
import bitsandbytes as bnb
import random

random_state = 1

random.seed(random_state)

In [2]:
def euclid(list1, list2):
    return sum((p-q)**2 for p, q in zip(list1, list2)) ** .5

In [3]:
with open('df_annotations_embeddings.pkl', 'rb') as f:
    df = pickle.load(f)
df

,Document,Split,Name,Id,Text,Type,Scheme,embeddings
0,A2008_Commission of the European Communities v...,1,prem,A1,must held first Article 4c CS prohibits granti...,L,Rule,"[-0.01507568359375, -1.73828125, 1.4287109375,..."
1,A2008_Commission of the European Communities v...,1,prem,A2,Also clear consistent caselaw Articles 4 CS 67...,L,"[Itpr, Prec, Rule]","[0.93017578125, -1.0146484375, 0.0575256347656..."
2,A2008_Commission of the European Communities v...,1,prem,A3,Court deduces Article 67 CS covers general mea...,L,"[Prec, Rule]","[0.195556640625, -1.734375, -0.1546630859375, ..."
3,A2008_Commission of the European Communities v...,1,prem,A4,Court also held action taken Article 67 CS can...,L,"[Prec, Rule]","[0.404296875, -0.2352294921875, 0.34326171875,..."
4,A2008_Commission of the European Communities v...,1,prem,A5,Court held particular inconceivable authors EC...,L,"[Itpr, Prec, Rule]","[1.0849609375, -0.98828125, 0.39453125, 0.8774..."
...,...,...,...,...,...,...,...,...
2530,R2021_World Duty Free v,5,prem,H7,question matter dialogue Spanish authorities C...,F,NaN,"[2.109375, -1.3642578125, -0.62744140625, 2.50..."
2531,R2021_World Duty Free v,5,prem,H8,present case action first instance WDFG compla...,F,NaN,"[0.880859375, -0.65966796875, -0.67626953125, ..."
2532,R2021_World Duty Free v,5,prem,H9,regard General Court recalled paragraph 212 ju...,F,NaN,"[2.0234375, -1.150390625, -0.70947265625, 1.59..."
2533,R2021_World Duty Free v,5,prem,H10,General Court err law concluding therefrom par...,F,NaN,"[1.6494140625, -1.5732421875, -0.857421875, 1...."


In [4]:
df_test = df[df['Split'] == 2]
df_test = df_test.dropna(subset = ['Scheme'])
df_test

,Document,Split,Name,Id,Text,Type,Scheme,embeddings
172,A2010_NDSHT Nya Destination Stockholm Hotell &...,2,prem,A2,regard must observed follows Article 58 Statut...,L,"[Prec, Rule]","[0.5068359375, -1.6494140625, -0.43408203125, ..."
177,A2010_NDSHT Nya Destination Stockholm Hotell &...,2,prem,B2,regard Court repeatedly held action annulment ...,L,"[Prec, Rule]","[1.076171875, -0.91064453125, -0.53271484375, ..."
178,A2010_NDSHT Nya Destination Stockholm Hotell &...,2,prem,B3,follows also settled caselaw concerning admiss...,L,Prec,"[0.90185546875, -1.087890625, -0.5048828125, 1..."
179,A2010_NDSHT Nya Destination Stockholm Hotell &...,2,prem,B4,contrast form act decision adopted principle i...,L,Itpr,"[0.92431640625, -0.471435546875, -0.3063964843..."
180,A2010_NDSHT Nya Destination Stockholm Hotell &...,2,prem,B5,therefore principle irrelevant classification ...,L,Itpr,"[0.7216796875, -0.82373046875, 0.39501953125, ..."
...,...,...,...,...,...,...,...,...
2123,R2017_European Commission v Frucona Košice a,2,prem,D13,connection also borne mind lawfulness decision...,L,Prec,"[0.83251953125, -0.8701171875, -0.16796875, 1...."
2124,R2017_European Commission v Frucona Košice a,2,prem,D14,However information ‘available’ Commission inc...,L,"[Itpr, Prec]","[1.4931640625, -1.1728515625, -0.70751953125, ..."
2126,R2017_European Commission v Frucona Košice a,2,prem,D16,Paragraphs 180 213 235 judgment appeal Commiss...,F,Itpr,"[0.55419921875, -1.5498046875, -0.5234375, 1.4..."
2130,R2017_European Commission v Frucona Košice a,2,prem,D20,Advocate General noted paragraphs 125 131 Opin...,F,Aut,"[1.2119140625, -1.08984375, 0.0924072265625, -..."


In [5]:
device = torch.device('cuda')
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [6]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16,
                  "quantization_config": {"load_in_4bit": True}},
    device_map="cuda:0"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [7]:
def test(template):

    count = 0
    autTP = 0
    autFN = 0
    autFP = 0
    classTP = 0
    classFN = 0
    classFP = 0
    itprTP = 0
    itprFN = 0
    itprFP = 0
    precTP = 0
    precFN = 0
    precFP = 0
    #princTP = 0
    #princFN = 0
    #princFP = 0
    ruleTP = 0
    ruleFN = 0
    ruleFP = 0

    for i in range(len(df_test)):

        query = 'Text: '+ df_test.iloc[i]['Text']+'\n'+'Answer: '

        
        print('query:\n'+ query,'\n')

        messages = [{'role': 'system', 'content': template},{"role": "user", "content": query}]

        outputs = pipeline(
            messages,
            max_new_tokens=10,
            do_sample = False
        )
        final_output = outputs[0]['generated_text'][-1]['content']
        print('LLM output:', final_output)
        print('ground truth:', df_test.iloc[i]['Scheme'])
        print(200*'-')

        if 'Aut' in final_output and 'Aut' in df_test.iloc[i]['Scheme']:
            autTP += 1
        if 'Aut' in final_output and 'Aut' not in df_test.iloc[i]['Scheme']:
            autFP += 1
        if 'Aut' not in final_output and 'Aut' in df_test.iloc[i]['Scheme']:
            autFN += 1
        
        if 'Class' in final_output and 'Class' in df_test.iloc[i]['Scheme']:
            classTP += 1
        if 'Class' in final_output and 'Class' not in df_test.iloc[i]['Scheme']:
            classFP += 1
        if 'Class' not in final_output and 'Class' in df_test.iloc[i]['Scheme']:
            classFN += 1

        if 'Itpr' in final_output and 'Itpr' in df_test.iloc[i]['Scheme']:
            itprTP += 1
        if 'Itpr' in final_output and 'Itpr' not in df_test.iloc[i]['Scheme']:
            itprFP += 1
        if 'Itpr' not in final_output and 'Itpr' in df_test.iloc[i]['Scheme']:
            itprFN += 1

        if 'Prec' in final_output and 'Prec' in df_test.iloc[i]['Scheme']:
            precTP += 1
        if 'Prec' in final_output and 'Prec' not in df_test.iloc[i]['Scheme']:
            precFP += 1
        if 'Prec' not in final_output and 'Prec' in df_test.iloc[i]['Scheme']:
            precFN += 1

        if 'Rule' in final_output and 'Rule' in df_test.iloc[i]['Scheme']:
            ruleTP += 1
        if 'Rule' in final_output and 'Rule' not in df_test.iloc[i]['Scheme']:
            ruleFP += 1
        if 'Rule' not in final_output and 'Rule' in df_test.iloc[i]['Scheme']:
            ruleFN += 1
            
    if autTP == 0:
        autF1 = 0
    else:
        autPREC = autTP/(autTP+autFP)
        autREC = autTP/(autTP+autFN)
        autF1 = 2/((1/autPREC)+(1/autREC))

    if classTP == 0:
        classF1 = 0
    else:
        classPREC = classTP/(classTP+classFP)
        classREC = classTP/(classTP+classFN)
        classF1 = 2/((1/classPREC)+(1/classREC))

    if itprTP == 0:
        itprF1 = 0
    else:
        itprPREC = itprTP/(itprTP+itprFP)
        itprREC = itprTP/(itprTP+itprFN)
        itprF1 = 2/((1/itprPREC)+(1/itprREC))

    if precTP == 0:
        precF1 = 0
    else:
        precPREC = precTP/(precTP+precFP)
        precREC = precTP/(precTP+precFN)
        precF1 = 2/((1/precPREC)+(1/precREC))

    if ruleTP == 0:
        ruleF1 = 0
    else:
        rulePREC = ruleTP/(ruleTP+ruleFP)
        ruleREC = ruleTP/(ruleTP+ruleFN)
        ruleF1 = 2/((1/rulePREC)+(1/ruleREC))

    print('Authoritative F1:', autF1)
    print('Verbal Classification F1:', classF1)
    print('Interpretation F1:', itprF1)
    print('Precedent F1:', precF1)
    print('Rule F1:', ruleF1)
    print()


    print('macro F1:', (autF1 + classF1 + itprF1 + precF1 + ruleF1)/5)
    print('macro F1 (reliable):', (autF1 + precF1 + ruleF1)/3)
            


In [8]:
template = '''Classify the following legal premise as one or more of the following argumentative schemes: Rule, Prec, Class, Itpr, Aut. Rule: whether there is an explicit or implicit reference to an article of law or citation of the text of a certain article. Prec: whether there is a reference to a previous ruling of the Supreme Court or the Court of Justice of the European Union. Class: if there is a definition of a legal concept or its constituent elements. Itpr: if there is reference to one of the interpretative criteria contained in Article 12 of the prelegislations (literal, teleological, psychological, systematic) to the Civil Code. Aut: if there is a reference to an indication by an authority (e.g. an opinion of the Advocate General). The expected output is a list with all applicable labels. For example: ['Prec', 'Aut', 'Rule']. Only reply with the list of labels.'''

In [9]:
test(template)

C:\Users\alfio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\alfio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


query:
Text: regard must observed follows Article 58 Statute Court Justice conjunction Article 1132 Rules Procedure Court Justice appeal appellant may put forward relevant argument provided subjectmatter proceedings General Court changed appeal Case C‑22905 P PKK KNK v Council 2007 ECR I‑439 paragraph 66 Case C‑806 P Herrero Romeu v Commission 2007 ECR I‑10333 paragraph 32
Answer:  



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


LLM output: ['Rule', 'Prec']
ground truth: ['Prec', 'Rule']
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
query:
Text: regard Court repeatedly held action annulment purposes Article 230 EC must available acts adopted institutions whatever nature form intended legal effects capable affecting interests applicant bringing distinct change legal position see inter alia Athinaïki Techniki v Commission paragraph 29 caselaw cited Case C‑36208 P Internationaler Hilfsfonds v Commission 2010 ECR I‑0000 paragraph 51
Answer:  



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

LLM output: ['Rule']
ground truth: ['Itpr', 'Prec']
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
query:
Text: Paragraphs 180 213 235 judgment appeal Commission’s objections referred paragraph 64 present judgment directed read legal context referred General Court imply new requirements incompatible caselaw Court Justice
Answer:  



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


LLM output: ['Prec', 'Rule']
ground truth: Itpr
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
query:
Text: Advocate General noted paragraphs 125 131 Opinion General Court merely noted paragraphs 191 195 198 199 judgment appeal internal contradictions decision issue made findings fact according none evidence administrative file able substantiate liquidation factors used Commission
Answer:  



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


LLM output: ['Aut', 'Prec']
ground truth: Aut
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
query:
Text: considerations far concern information normally prudent diligent private creditor situation close possible local tax office could priori ignore capable justifying General Court’s decision Commission failed take consideration relevant information see effect judgment 24 January 2013 Frucona Košice v Commission C‑7311 P EUC201332 paragraphs 77 78 81
Answer:  

LLM output: ['Prec', 'Rule']
ground truth: Prec
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Authoritative F1: 0.6666666666666666
Verbal Classification F1: 0
Interpretation F1: 0.22857142857142856
Precedent F1: 0.84444444444